In [18]:
from dotenv import load_dotenv
import os

load_dotenv()

if os.getenv("GROQ_API_KEY"):
    print('Groq API Key loaded successfully')
else:
    print('Groq API Key loading failed, please make sure the .env file exists and the spelling is correct')

if os.getenv("TAVILY_API_KEY"):
    print('Tavily API Key loaded successfully')
else:
    print('Tavily API Key loading failed, please make sure the .env file exists and the spelling is correct')

Groq API Key loaded successfully
Tavily API Key loaded successfully


In [19]:
# read file wordlist-eng.txt
wordlist = open("wordlist-eng.txt", "r")
words = wordlist.readlines()
wordlist.close()

# remove newline characters
words = [word.strip() for word in words]


print(words)

FileNotFoundError: [Errno 2] No such file or directory: 'wordlist-eng.txt'

In [ ]:
import re
import httpx
import os

from langchain_groq import ChatGroq
load_dotenv()

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
import random


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_groq import ChatGroq

In [ ]:
class SpymasterState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    clue: str
    n_words: int

class OperativeState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    clue: str
    guess_number: int
    guesses: Annotated[list[str], operator.add]
    last_guess_correct: bool

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
class Board:
    def __init__(self,word_list,starting_team,random_state=42):
        random.seed(random_state)
        self.random_state = random_state
        self.word_list = word_list
        self.starting_team = starting_team

        self._prepare_grid()

    def _prepare_grid(self):
        words = random.sample(self.word_list, 25)

        n_words_team_R = 9 if self.starting_team == "R" else 8
        n_words_team_B = 9 if self.starting_team == "B" else 8
        n_words_bomb = 1
        chosen_words = random.sample(words, n_words_team_R + n_words_team_B + n_words_bomb)

        words_team_R = chosen_words[:n_words_team_R]
        words_team_B = chosen_words[n_words_team_R:n_words_team_R + n_words_team_B]
        words_bomb = chosen_words[n_words_team_R + n_words_team_B:]

        self.grid = {
            word: {
                "team":"R" if word in words_team_R else "B" if word in words_team_B else "X" if word in words_bomb else "N",
                "revealed":False
                } 
            for word in words
            }
        self.grid_hidden = {word: {"team":"?","revealed":False} for word in words}

    def get_grid(self):
        return self.grid
    
    def get_hidden_grid(self):
        self._update_hidden_grid()
        return self.grid_hidden
        
    def reveal_word(self,word):
        self.grid[word]["revealed"] = True
        self._update_hidden_grid()
        print(f"Player revealed word: {word}")
        return self.grid[word]["team"] 
    
    def give_clue(self,clue,n_words):
        print(f"Player gives clue: {clue} for {n_words} words")
    
    def _update_hidden_grid(self):
        for word in self.grid:
            if self.grid[word]["revealed"]:
                self.grid_hidden[word]["team"] = self.grid[word]["team"]
                self.grid_hidden[word]["revealed"] = True

In [ ]:
class Spymaster:
    def __init__(self, model, prompt_func, grid: Board, team):
        self.model = model
        # prepare the StateGraph
        graph = StateGraph(SpymasterState)
        # create the nodes of the graph and corresponding behaviour
        graph.add_node("head", self.create_clue)
        graph.set_entry_point("head")
        # create the graph and turn it into a LangChain Runnable (allows to call methods like .invoke,...)
        self.graph = graph.compile()
        self.prompt_func = prompt_func
        self.team=team
        self.grid = grid

    # create a clue (all in one -> expand later on)
    def create_clue(self, state: SpymasterState):
        messages = [SystemMessage(content=self.prompt_func(self.team))] + [SystemMessage(str(self.grid.get_grid()))]
        retries_bad_answer = 5
        retries_model_failed = 5
        for _ in range(retries_bad_answer):
            for _ in range(retries_model_failed):
                message = self.model.invoke(messages)
                if message.content != "":
                    break
            if message.content == "":
                raise Exception("Model failed to generate a message after {retries_model_failed} retries")
            clue, n_words = self.extract_clue(message.content)
            if clue is None or n_words is None:
                print("Model failed to provide a valid clue")
                if message.content == "":
                    messages.append(SystemMessage(content=f"Your guess was not in the correct format. Use the format '-[<clue>,<number>]-'"))
            else:
                break
                


        
        return {
            'messages': [message],
            'clue': clue,
            'n_words': n_words}
    
    def extract_clue(self, message):
        # extract clue and number of words from message using regex of the form "-[<clue>,<number>]-"
        pattern = r"-\[(.*),(\d+)\]-"
        match = re.search(pattern, message)
        print(message)
        if match:
            return match.group(1), int(match.group(2))
        else:
            return None, None

In [ ]:
class Operative:
    def __init__(self, model, system_prompt_planner, system_prompt_guesser, grid: Board, team, clue, n_words):
        self.model = model
        # set the system message
        self.system_prompt_planner = system_prompt_planner
        self.system_prompt_guesser = system_prompt_guesser
        # set the grid
        self.grid = grid
        # set the team
        self.team = team

        self.clue = clue
        self.n_words = n_words

        # prepare the StateGraph
        graph = StateGraph(OperativeState)
        # create the nodes of the graph and corresponding behaviour
        graph.add_node("planner", self.planner)
        graph.add_node("prepare_guess", self.prepare_guess)
        graph.add_edge("planner", "prepare_guess")
        graph.add_conditional_edges("prepare_guess", self.new_guess_available, {True: "prepare_guess", False: END})
        
        graph.set_entry_point("planner")
        # create the graph and turn it into a LangChain Runnable (allows to call methods like .invoke,...)
        self.graph = graph.compile()

    # plan the round
    def planner(self, state: OperativeState):
        new_info = f"""
        The clue is: {self.clue} with {self.n_words} words. The grid is: {self.grid.get_hidden_grid()}. Assign the a percentage of confidence to each word."""
        
        messages = [SystemMessage(content=self.system_prompt_planner)] + [SystemMessage(content=new_info)]
        retries_model_failed = 5
        for _ in range(retries_model_failed):
            message = self.model.invoke(messages)
            if message.content == "":
                print("Empty message, retrying...")
            else:
                break
            
        return {'messages': [SystemMessage(content=new_info)] + [message]}
    
    # create a guess (all in one -> expand later on)
    def prepare_guess(self, state: OperativeState):
        messages = [SystemMessage(content=self.system_prompt_guesser)] + state['messages']
        retries_model_failed = 5
        retries_bad_answer = 5
        for _ in range(retries_bad_answer):
            for _ in range(retries_model_failed): # number how often the model will retry to generate a message
                message = self.model.invoke(messages)
                if message.content == "":
                    print("Empty message, retrying...")
                else:
                    break
            if message.content == "":
                raise Exception(f"Model failed to generate a message after {retries_model_failed} retries")
            guess, errors = self.extract_guess(message.content)
            # check if guess is None
            if guess is None:
                print("Invalid guess, retrying...")
                messages.append(SystemMessage(content=f"Your last guess was invalid for the following reasons: {str(errors)}."))
            else:
                break
        
        if guess is None:
            print("Model unable to generate a valid guess, ending turn.")
            guess = "!END!"

        try:
            guess_number = state['guesses'] + 1
        except:
            guess_number = 1

        return {
            'messages': [message],
            'guesses': [guess],
            'guess_number': guess_number}
    
    def take_guess(self, state: OperativeState):
        guess = state['guesses'][-1]
        print(f"taking guess {guess=}")
        team = self.grid.reveal_word(guess)
        return team == self.team # return True if the guess was correct, False otherwise
    
    def new_guess_available(self, state: OperativeState):
        if state['guesses'][-1] == "!END!":
            return False
        
        guess_correct = self.take_guess(state)
        if not guess_correct:
            return False
        if state['guess_number'] == self.n_words+1:
            return False
        return True

    def extract_guess(self, message):
        # extract guess from message using regex of the form "-[<guess>]-"; for ending the turn it should be "[guess: !END!]"
        pattern = r'-\[[A-Z]+\]-'
        print(f"{message=}")
        match = re.search(pattern, message)
        if match:
            print(f"{match=}")
            guess = match.group(0)[2:-2]
            guess_valid = self.guess_valid(guess)
            if guess_valid[0] and guess_valid[1] and guess_valid[2]:
                errors = []
            else:
                guess = None
                errors = []
                if not guess_valid[0]:
                    errors.append("Not in grid")
                if not guess_valid[1]:
                    errors.append("Already revealed")
                if not guess_valid[2]:
                    errors.append("Not correct team")

        else:
            guess = None
            errors = ["Wrong format"]

        return guess, errors
    
    def guess_valid(self, guess):
        in_grid = guess in self.grid.get_grid()
        not_revealed = not self.grid.get_grid()[guess]["revealed"]
        correct_team = self.grid.get_grid()[guess]["team"] == self.team
        return in_grid, not_revealed, correct_team

In [ ]:
def get_prompt_spymaster(team):
    prompt = f""" You take a role in a game and play for team {team}.
    You are given a list of words. For each word, you will see which team it belongs to and wether it was already revealed to your counterpart.
    Your task is to find words, that belong to your team, were not revealed yet and share some similiarities. You then need to find a word that connects these words. This is called clue. You also need to provide the number of words, you tried to connect with your clue. Give your answer in the format: -[<clue>,<number>]-.
    """
    return prompt

def get_prompt_operative_planner(team):
    prompt = f"""You take a role in a game and play for team {team}.
    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number. Using this clue assign probabilities to each word of the list, that has not yet been revealed, that the clue is related to it. 
    """
    return prompt

def get_prompt_operative_prepareGuess(team):
    prompt = f"""You take a role in a game and play for team {team}.
    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number. Another player has assigned probabilities to each word of the list, that has not yet been revealed, that the clue is related to it. You can guess one word at a time. If you you are correct, you can continue guessing. If you are wrong, your turn ends. 
    You can guess by providing a word in the format -[<guess>]-.
    If you are unsure, you can end your turn by guessing "!END!".
    """
    return prompt

In [ ]:
class Match:
    def __init__(self,words, starting_team="R"):

        self.grid = Board(words, starting_team)

        self.current_team = starting_team # R or B
        self.current_role = "spymaster" # spymaster or operative

        self.spymaster_red = Spymaster(chat_model, get_prompt_spymaster(self.current_team), self.grid, "R")
        self.operative_red = Operative(chat_model, get_prompt_operative_planner("R"), get_prompt_operative_prepareGuess("R"), self.grid, "R", "clue", "R")
        self.spymaster_blue = Spymaster(chat_model, get_prompt_spymaster(self.current_team), self.grid, "B")
        self.operative_blue = Operative(chat_model, get_prompt_operative_planner("B"), get_prompt_operative_prepareGuess("B"), self.grid, "B", "clue", "B")

        winner = self.game_loop()
        print(f"Team {winner} wins!")


    def game_loop(self):
        while True:
            # turn of spymaster
            messages = [SystemMessage(content=f"The grid is: {self.grid.get_grid()}. Create a clue for your team.")]
            if self.current_team == "R":
                result = self.spymaster_red.graph.invoke({"messages": messages})
            else:
                result = self.spymaster_blue.graph.invoke({"messages": messages})
            clue = result['clue']
            n_words = result['n_words']
            print(f"Team {self.current_team} spymaster gives clue: {clue} for {n_words}")

            # turn of operative
            messages = [SystemMessage(content=f"You are playing for team {self.current_team}. The grid is: {self.grid.get_hidden_grid()}. The clue is: {clue} for {n_words} words.")]
            if self.current_team == "R":
                result = self.operative_red.graph.invoke({"messages": messages})
            else:
                result = self.operative_blue.graph.invoke({"messages": messages})

            guesses = result['guesses']
            print(f"Team {self.current_team} operative guessed: {guesses}")

            # check if game is over
            game_over = self.check_game_over(self.current_team)
            if game_over:
                return game_over # representing the winning team
            
            # switch players
            if self.current_role == "operative":
                self.current_team = "R" if self.current_team == "B" else "B"
            self.current_role = "operative" if self.current_role == "spymaster" else "spymaster"
            
    def check_game_over(self, current_team):
        team_R_revealed = [self.grid.grid[word]["revealed"] for word in self.grid.grid if self.grid.grid[word]["team"] == "R"]
        team_B_revealed = [self.grid.grid[word]["revealed"] for word in self.grid.grid if self.grid.grid[word]["team"] == "B"]
        bomb_revealed = [self.grid.grid[word]["revealed"] for word in self.grid.grid if self.grid.grid[word]["team"] == "X"]

        if all(team_R_revealed):
            return "R"
        elif all(team_B_revealed):
            return "R"
        elif any(bomb_revealed):
            return not current_team # the other team wins if bomb was revealed
        else:
            return False

In [ ]:
match = Match(words)

After analyzing the given list of words, I found some words that belong to my team (team 1), were not revealed yet, and share some similarities. 

The words I found are: MEXICO, AUSTRALIA, and BUFFALO. These words are all names of countries or places.

A clue that connects these words is: COUNTRY.

I tried to connect 3 words with my clue.

My answer is: -[COUNTRY,3]-
After analyzing the given list of words, I found some words that belong to my team (team 1), were not revealed yet, and share some similarities. 

The words I found are: MEXICO, AUSTRALIA, and BUFFALO. These words are all names of countries or places.

A clue that connects these words is: COUNTRY.

I tried to connect 3 words with my clue.

My answer is: -[COUNTRY,3]-
Team 1 spymaster gives clue: COUNTRY for 3
messages: [SystemMessage(content='You take a role in a game and play for team 1.\n    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number

KeyError: None